In [1]:
# Importing the relevant packages
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins.hparams import api as hp

## Downloading and preprocessing the data

In [2]:
# Defining some constants/hyperparameters
BUFFER_SIZE = 70_000 # for reshuffling
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
# Downloading the MNIST dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
# Creating a function to scale our data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

In [6]:
# Scaling the data
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
# Defining the size of the validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
# Defining the size of the test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
# Reshuffling the dataset
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
# Splitting the dataset into trainig + validation
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
# Batching the data
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = mnist_test.map(scale).batch(num_test_samples)

## Defining hyperparameters

In [12]:
# Defining the hypermatarest we would test and their range
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## Cerating functions for training our model and for logging purposes

In [13]:
# Wrapping our model and training in a function
def train_test_model(hparams):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=loss_fn, metrics=['accuracy'])
    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_data, 
        epochs = NUM_EPOCHS,
        callbacks = [early_stopping],
        validation_data = validation_data,
        verbose = 2
    )
    
    _, accuracy = model.evaluate(test_data)
    
    return accuracy

In [14]:
# Creating a function to log the resuls
def run(log_dir, hparams):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

## Training the model with the different hyperparameters

In [15]:
# Performing a grid search on the hyperparameters we need to test
session_num = 0

for filter_size in HP_FILTER_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
    
        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_OPTIMIZER: optimizer
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)

        session_num += 1

--- Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}
Epoch 1/20
422/422 - 21s - loss: 0.2906 - accuracy: 0.9168 - val_loss: 0.0817 - val_accuracy: 0.9762
Epoch 2/20
422/422 - 20s - loss: 0.0801 - accuracy: 0.9754 - val_loss: 0.0600 - val_accuracy: 0.9830
Epoch 3/20
422/422 - 21s - loss: 0.0606 - accuracy: 0.9820 - val_loss: 0.0520 - val_accuracy: 0.9848
Epoch 4/20
422/422 - 21s - loss: 0.0496 - accuracy: 0.9850 - val_loss: 0.0389 - val_accuracy: 0.9882
Epoch 5/20
422/422 - 21s - loss: 0.0423 - accuracy: 0.9873 - val_loss: 0.0293 - val_accuracy: 0.9907
Epoch 6/20
422/422 - 20s - loss: 0.0370 - accuracy: 0.9887 - val_loss: 0.0266 - val_accuracy: 0.9935
Epoch 7/20
422/422 - 20s - loss: 0.0310 - accuracy: 0.9904 - val_loss: 0.0251 - val_accuracy: 0.9930
Epoch 8/20
422/422 - 21s - loss: 0.0276 - accuracy: 0.9915 - val_loss: 0.0220 - val_accuracy: 0.9932
Epoch 9/20
422/422 - 20s - loss: 0.0243 - accuracy: 0.9925 - val_loss: 0.0154 - val_accuracy: 0.9955
Epoch 10/20
422/422 - 21s

Epoch 11/20
422/422 - 20s - loss: 0.1082 - accuracy: 0.9683 - val_loss: 0.1077 - val_accuracy: 0.9710
Epoch 12/20
422/422 - 20s - loss: 0.1022 - accuracy: 0.9702 - val_loss: 0.0974 - val_accuracy: 0.9718
Epoch 13/20
422/422 - 20s - loss: 0.0969 - accuracy: 0.9713 - val_loss: 0.0961 - val_accuracy: 0.9710
Epoch 14/20
422/422 - 21s - loss: 0.0935 - accuracy: 0.9728 - val_loss: 0.0795 - val_accuracy: 0.9780
Epoch 15/20
422/422 - 21s - loss: 0.0893 - accuracy: 0.9740 - val_loss: 0.0865 - val_accuracy: 0.9742
Epoch 16/20
422/422 - 20s - loss: 0.0847 - accuracy: 0.9756 - val_loss: 0.0802 - val_accuracy: 0.9750
      1/Unknown - 1s 899ms/step - loss: 0.0812 - accuracy: 0.975 - 1s 910ms/step - loss: 0.0812 - accuracy: 0.9756

## Visualizing the hyperparameter results with Tensorboard

In [2]:
# Loading the Tensorboard extension
%load_ext tensorboard
%tensorboard --logdir "logs/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3304), started 0:00:59 ago. (Use '!kill 3304' to kill it.)